In [2]:
import numpy as np
import pandas as pa
letters_train = pa.read_csv(
    'https://www.labri.fr/perso/zemmari/datasets/emnist/emnist-letters-train.csv',
    header=None)
letters_test = pa.read_csv(
    'https://www.labri.fr/perso/zemmari/datasets/emnist/emnist-letters-test.csv',
    header=None)
def load_letters():
    X_train = np.array(letters_train.iloc[:, 1:785])
    y_train = np.array(letters_train.iloc[:, 0])
    y_train = y_train - 1
    X_test = np.array(letters_test.iloc[:, 1:785])
    y_test = np.array(letters_test.iloc[:, 0])
    y_test = y_test -1 
    X_train = np.reshape(X_train, (X_train.shape[0], 28, 28))
    X_test = np.reshape(X_test, (X_test.shape[0], 28, 28))
    return (X_train, y_train), (X_test, y_test)

In [35]:
# Question 1
import tensorflow as tf
import keras
from keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = load_letters()
img_rows, img_cols = X_train.shape[1], X_train.shape[2]

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train  = X_train / 255
X_test  = X_test / 255

unique_classes = np.unique(y_train)
print(unique_classes)

num_classes = unique_classes.size


y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
X_train shape: (88800, 28, 28, 1)
y_train shape: (88800, 26)
X_test shape: (14800, 28, 28, 1)
y_test shape: (14800, 26)


In [36]:
# Question 2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def letter_network():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(26, activation='softmax')) #26 lettres dans l'alphabet
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = letter_network()
model.summary()
history = model.fit(X_train, y_train, epochs=10, batch_size=100, validation_data=(X_test, y_test))
scores = model.evaluate(X_test, y_test)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 1600)              0         
                                                                 
 dense_18 (Dense)            (None, 128)             

KeyboardInterrupt: 